In [2]:
{
 "cells": [
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "Import all the packages we need",
   "id": "1c8963d05f057271"
  },
  {
   "cell_type": "code",
   "id": "initial_id",
   "metadata": {
    "collapsed": true,
    "ExecuteTime": {
     "end_time": "2025-03-17T16:46:21.689954Z",
     "start_time": "2025-03-17T16:46:21.672434Z"
    }
   },
   "source": [
    "from typing import List\n",
    "\n",
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "import h5py\n",
    "import os\n",
    "tensorflow.python.keras.utils.np_utils import to_categorical\n",
    "from keras.src.backend.tensorflow.sparse import zeros_int8\n",
    "from tensorflow import Tensor\n",
    "from tensorflow import Tensor\n",
    "from tensorflow.keras import layers\n",
    "from tensorflow.keras import Model\n",
    "from tensorflow.keras.optimizers import Adam\n",
    "from typing import Tuple, Dict, Any, Sequence\n",
    "\n",
    "from collections import defaultdict\n",
    "import tensorflow as tf\n",
    "\n"
   ],
   "outputs": [],
   "execution_count": 4
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "",
   "id": "3d74052233fbb31b"
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "Import the datasets, and extract the information we are going to use (sub_byte_in and byte 0 in our test case)\n",
   "id": "ed066cdcf51960f"
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:28:12.700089Z",
     "start_time": "2025-03-13T14:28:10.319752Z"
    }
   },
   "cell_type": "code",
   "source": [
    "#start with train\n",
    "dataset_name = \"train\"\n",
    "shard = []\n",
    "attack_byte = 0\n",
    "attack_point = \"sub_bytes_in\"\n",
    "max_trace_length = 5000\n",
    "num_traces = 256\n",
    "num_traces_per_shard = 256\n",
    "full_key = False\n",
    "x_List: List[Tensor] = []\n",
    "y_List: List[Tensor] = []\n",
    "\n",
    "#need to open the file to read any info\n",
    "with (h5py.File(f\"{dataset_name}.hdf5\", \"r\")) as f:\n",
    "    for group in f.keys():\n",
    "        group_name = f[group]\n",
    "\n",
    "        x_shard = group_name[\"traces\"][:num_traces_per_shard, :5000, :]\n",
    "        x_shard = tf.convert_to_tensor(x_shard, dtype=\"float32\")\n",
    "\n",
    "        y_shard = group_name[\"sub_bytes_in\"][attack_byte]\n",
    "        y_shard = y_shard[:num_traces_per_shard]\n",
    "        y_shard = tf.keras.utils.to_categorical(y_shard, 256)\n",
    "        y_shard = tf.convert_to_tensor(y_shard, dtype=\"uint8\")\n",
    "\n",
    "\n",
    "        x_List.append(x_shard)\n",
    "        y_List.append(y_shard)\n",
    "    x: Tensor = tf.concat(x_List, axis=0)\n",
    "    y: Tensor = tf.concat(y_List, axis=0)\n",
    "x_train = x\n",
    "y_train = y"
   ],
   "id": "7947aceee98f4918",
   "outputs": [],
   "execution_count": 71
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:28:15.821410Z",
     "start_time": "2025-03-13T14:28:12.715078Z"
    }
   },
   "cell_type": "code",
   "source": [
    "#For test dataset, which should be scaled down as it is 1:1 to the train set atm\n",
    "dataset_name = \"test\"\n",
    "shard = []\n",
    "attack_byte = 0\n",
    "attack_point = \"sub_bytes_in\"\n",
    "max_trace_length = 5000\n",
    "num_traces = 256 \n",
    "num_traces_per_shard = 256\n",
    "full_key = False\n",
    "x_List: List[Tensor] = []\n",
    "y_List: List[Tensor] = []\n",
    "\n",
    "#need to open the file to read any info\n",
    "with (h5py.File(f\"{dataset_name}.hdf5\", \"r\")) as f:\n",
    "    for group in f.keys():\n",
    "        group_name = f[group]\n",
    "\n",
    "        x_shard = group_name[\"traces\"][:num_traces_per_shard, :5000, :]\n",
    "        x_shard = tf.convert_to_tensor(x_shard, dtype=\"float32\")\n",
    "\n",
    "        y_shard = group_name[\"sub_bytes_in\"][attack_byte]\n",
    "        y_shard = y_shard[:num_traces_per_shard]\n",
    "        y_shard = tf.keras.utils.to_categorical(y_shard, 256)\n",
    "        y_shard = tf.convert_to_tensor(y_shard, dtype=\"uint8\")\n",
    "\n",
    "\n",
    "        x_List.append(x_shard)\n",
    "        y_List.append(y_shard)\n",
    "    x: Tensor = tf.concat(x_List, axis=0)\n",
    "    y: Tensor = tf.concat(y_List, axis=0)\n",
    "x_test = x\n",
    "y_test = y"
   ],
   "id": "f372935e82190601",
   "outputs": [],
   "execution_count": 72
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": [
    "Create the resnet model we want to train(cannot use pretrained model from scaaml as it uses depreciated methods and just wont work when we try) but we use the structure\n",
    "\n"
   ],
   "id": "7f15bd199fca45f9"
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:28:15.846916Z",
     "start_time": "2025-03-13T14:28:15.835887Z"
    }
   },
   "cell_type": "code",
   "source": [
    "def block(x: Tensor,\n",
    "          filters: int,\n",
    "          kernel_size: int = 3,\n",
    "          strides: int = 1,\n",
    "          conv_shortcut: bool = False,\n",
    "          activation: str = \"relu\") -> Tensor:\n",
    "    \"\"\"Residual block with pre-activation\n",
    "    From: https://arxiv.org/pdf/1603.05027.pdf\n",
    "\n",
    "    Args:\n",
    "        x: input tensor.\n",
    "        filters (int): filters of the bottleneck layer.\n",
    "\n",
    "        kernel_size(int, optional): kernel size of the bottleneck layer.\n",
    "        defaults to 3.\n",
    "\n",
    "        strides (int, optional): stride of the first layer.\n",
    "        defaults to 1.\n",
    "\n",
    "        conv_shortcut (bool, optional): Use convolution shortcut if True,\n",
    "        otherwise identity shortcut. Defaults to False.\n",
    "\n",
    "        use_batchnorm (bool, optional): Use batchnormalization if True.\n",
    "        Defaults to True.\n",
    "\n",
    "        activation (str, optional): activation function. Defaults to \"relu\".\n",
    "\n",
    "    Returns:\n",
    "        Output tensor for the residual block.\n",
    "    \"\"\"\n",
    "\n",
    "    x = layers.BatchNormalization()(x)\n",
    "    x = layers.Activation(activation)(x)\n",
    "\n",
    "    if conv_shortcut:\n",
    "        shortcut = layers.Conv1D(4 * filters, 1, strides=strides)(x)\n",
    "    else:\n",
    "        if strides > 1:\n",
    "            shortcut = layers.MaxPooling1D(1, strides=strides)(x)\n",
    "        else:\n",
    "            shortcut = x\n",
    "\n",
    "    x = layers.Conv1D(filters, 1, use_bias=False, padding=\"same\")(x)\n",
    "    x = layers.BatchNormalization()(x)\n",
    "    x = layers.Activation(activation)(x)\n",
    "\n",
    "    x = layers.Conv1D(filters,\n",
    "                      kernel_size,\n",
    "                      strides=strides,\n",
    "                      use_bias=False,\n",
    "                      padding=\"same\")(x)\n",
    "    x = layers.BatchNormalization()(x)\n",
    "    x = layers.Activation(activation)(x)\n",
    "\n",
    "    x = layers.Conv1D(4 * filters, 1)(x)\n",
    "    x = layers.Add()([shortcut, x])\n",
    "    return x\n",
    "\n",
    "\n",
    "# pylint: disable=too-many-positional-arguments\n",
    "def stack(x: Tensor,\n",
    "          filters: int,\n",
    "          blocks: int,\n",
    "          kernel_size: int = 3,\n",
    "          strides: int = 2,\n",
    "          activation: str = \"relu\") -> Tensor:\n",
    "    \"\"\"A set of stacked residual blocks.\n",
    "    Args:\n",
    "        filters (int): filters of the bottleneck layer.\n",
    "\n",
    "        blocks (int): number of conv blocks to stack.\n",
    "\n",
    "        kernel_size(int, optional): kernel size of the bottleneck layer.\n",
    "        defaults to 3.\n",
    "\n",
    "        strides (int, optional): stride used in the last block.\n",
    "        defaults to 2.\n",
    "\n",
    "        conv_shortcut (bool, optional): Use convolution shortcut if True,\n",
    "        otherwise identity shortcut. Defaults to False.\n",
    "\n",
    "        activation (str, optional): activation function. Defaults to \"relu\".\n",
    "\n",
    "    Returns:\n",
    "        tensor:Output tensor for the stacked blocks.\n",
    "  \"\"\"\n",
    "    x = block(x,\n",
    "              filters,\n",
    "              kernel_size=kernel_size,\n",
    "              activation=activation,\n",
    "              conv_shortcut=True)\n",
    "    for _ in range(2, blocks):\n",
    "        x = block(x, filters, kernel_size=kernel_size, activation=activation)\n",
    "    x = block(x, filters, strides=strides, activation=activation)\n",
    "    return x\n",
    "\n",
    "#endrer fra Model[Any, Any] til bare model\n",
    "# pylint: disable=C0103\n",
    "def Resnet1D(input_shape: Tuple[int, ...], attack_point: str):\n",
    "    #mdl_cfg: Dict[str, Any], optim_cfg: Dict[str, Any]) -> Model:\n",
    "    del attack_point  # unused\n",
    "\n",
    "    #pool_size = mdl_cfg[\"initial_pool_size\"]\n",
    "    #filters = mdl_cfg[\"initial_filters\"]\n",
    "    #block_kernel_size = mdl_cfg[\"block_kernel_size\"]\n",
    "   # activation = mdl_cfg[\"activation\"]\n",
    "    #dense_dropout = mdl_cfg[\"dense_dropout\"]\n",
    "    #num_blocks = [\n",
    "    #    mdl_cfg[\"blocks_stack1\"], mdl_cfg[\"blocks_stack2\"],\n",
    "   #     mdl_cfg[\"blocks_stack3\"], mdl_cfg[\"blocks_stack4\"]\n",
    "  #  ]\n",
    "#manually adding the parameters as i have not figured out how to read json into the model properly yet.\n",
    "    pool_size = 4\n",
    "    filters = 8\n",
    "    block_kernel_size = 3\n",
    "    activation = \"relu\"\n",
    "    dense_dropout = 0.1\n",
    "    num_blocks = [\n",
    "        3, 4,\n",
    "        4, 3\n",
    "    ]\n",
    "\n",
    "    inputs = layers.Input(shape=input_shape)\n",
    "    x = inputs\n",
    "\n",
    "    # stem\n",
    "    x = layers.MaxPool1D(pool_size=pool_size)(x)\n",
    "\n",
    "    # trunk: stack of residual block\n",
    "    for block_idx in range(4):\n",
    "        filters *= 2\n",
    "        x = stack(x,\n",
    "                  filters,\n",
    "                  num_blocks[block_idx],\n",
    "                  kernel_size=block_kernel_size,\n",
    "                  activation=activation)\n",
    "\n",
    "    # head model: dense\n",
    "    x = layers.GlobalAveragePooling1D()(x)\n",
    "    for _ in range(1):\n",
    "        x = layers.Dropout(dense_dropout)(x)\n",
    "        x = layers.Dense(256)(x)\n",
    "        x = layers.BatchNormalization()(x)\n",
    "        x = layers.Activation(activation)(x)\n",
    "\n",
    "    outputs = layers.Dense(256, activation=\"softmax\")(x)\n",
    "\n",
    "    model: Model[Any, Any] = Model(inputs=inputs, outputs=outputs)\n",
    "    model.summary()\n",
    "\n",
    "    #if get_num_gpu() > 1:\n",
    "        #lr = optim_cfg[\"multi_gpu_lr\"]\n",
    "        #lr = 0.001\n",
    "\n",
    "    #else:\n",
    "        #lr = optim_cfg[\"lr\"]\n",
    "    lr = 0.001\n",
    "    model.compile(loss=[\"categorical_crossentropy\"],\n",
    "                  metrics=[\"acc\"],\n",
    "                  optimizer=Adam(lr))\n",
    "    return model"
   ],
   "id": "399a2bc4218780fc",
   "outputs": [],
   "execution_count": 73
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:28:15.899610Z",
     "start_time": "2025-03-13T14:28:15.861787Z"
    }
   },
   "cell_type": "code",
   "source": "",
   "id": "acb4b5e8e19eac2a",
   "outputs": [
    {
     "ename": "FileNotFoundError",
     "evalue": "[Errno 2] No such file or directory: 'notebooks\\\\stm32f0_config.json'",
     "output_type": "error",
     "traceback": [
      "\u001B[31m---------------------------------------------------------------------------\u001B[39m",
      "\u001B[31mFileNotFoundError\u001B[39m                         Traceback (most recent call last)",
      "\u001B[36mCell\u001B[39m\u001B[36m \u001B[39m\u001B[32mIn[74]\u001B[39m\u001B[32m, line 4\u001B[39m\n\u001B[32m      1\u001B[39m \u001B[38;5;28;01mimport\u001B[39;00m\u001B[38;5;250m \u001B[39m\u001B[34;01mjson\u001B[39;00m\n\u001B[32m----> \u001B[39m\u001B[32m4\u001B[39m \u001B[38;5;28;01mwith\u001B[39;00m \u001B[38;5;28;43mopen\u001B[39;49m\u001B[43m(\u001B[49m\u001B[33;43m\"\u001B[39;49m\u001B[33;43mnotebooks\u001B[39;49m\u001B[38;5;130;43;01m\\\\\u001B[39;49;00m\u001B[33;43mstm32f0_config.json\u001B[39;49m\u001B[33;43m\"\u001B[39;49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[33;43m\"\u001B[39;49m\u001B[33;43mr\u001B[39;49m\u001B[33;43m\"\u001B[39;49m\u001B[43m)\u001B[49m \u001B[38;5;28;01mas\u001B[39;00m f:\n\u001B[32m      5\u001B[39m     config = json.load(f)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\IPython\\core\\interactiveshell.py:325\u001B[39m, in \u001B[36m_modified_open\u001B[39m\u001B[34m(file, *args, **kwargs)\u001B[39m\n\u001B[32m    318\u001B[39m \u001B[38;5;28;01mif\u001B[39;00m file \u001B[38;5;129;01min\u001B[39;00m {\u001B[32m0\u001B[39m, \u001B[32m1\u001B[39m, \u001B[32m2\u001B[39m}:\n\u001B[32m    319\u001B[39m     \u001B[38;5;28;01mraise\u001B[39;00m \u001B[38;5;167;01mValueError\u001B[39;00m(\n\u001B[32m    320\u001B[39m         \u001B[33mf\u001B[39m\u001B[33m\"\u001B[39m\u001B[33mIPython won\u001B[39m\u001B[33m'\u001B[39m\u001B[33mt let you open fd=\u001B[39m\u001B[38;5;132;01m{\u001B[39;00mfile\u001B[38;5;132;01m}\u001B[39;00m\u001B[33m by default \u001B[39m\u001B[33m\"\u001B[39m\n\u001B[32m    321\u001B[39m         \u001B[33m\"\u001B[39m\u001B[33mas it is likely to crash IPython. If you know what you are doing, \u001B[39m\u001B[33m\"\u001B[39m\n\u001B[32m    322\u001B[39m         \u001B[33m\"\u001B[39m\u001B[33myou can use builtins\u001B[39m\u001B[33m'\u001B[39m\u001B[33m open.\u001B[39m\u001B[33m\"\u001B[39m\n\u001B[32m    323\u001B[39m     )\n\u001B[32m--> \u001B[39m\u001B[32m325\u001B[39m \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[43mio_open\u001B[49m\u001B[43m(\u001B[49m\u001B[43mfile\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43m*\u001B[49m\u001B[43margs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43m*\u001B[49m\u001B[43m*\u001B[49m\u001B[43mkwargs\u001B[49m\u001B[43m)\u001B[49m\n",
      "\u001B[31mFileNotFoundError\u001B[39m: [Errno 2] No such file or directory: 'notebooks\\\\stm32f0_config.json'"
     ]
    }
   ],
   "execution_count": 74
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:28:35.191330Z",
     "start_time": "2025-03-13T14:28:34.519160Z"
    }
   },
   "cell_type": "code",
   "source": "sub_in_0_byte_cnn_scaaml = Resnet1D((5000, 1), \"sub_bytes_in\")",
   "id": "1090dd3837d4ef39",
   "outputs": [
    {
     "data": {
      "text/plain": [
       "\u001B[1mModel: \"functional_2\"\u001B[0m\n"
      ],
      "text/html": [
       "<pre style=\"white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace\"><span style=\"font-weight: bold\">Model: \"functional_2\"</span>\n",
       "</pre>\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓\n",
       "┃\u001B[1m \u001B[0m\u001B[1mLayer (type)       \u001B[0m\u001B[1m \u001B[0m┃\u001B[1m \u001B[0m\u001B[1mOutput Shape     \u001B[0m\u001B[1m \u001B[0m┃\u001B[1m \u001B[0m\u001B[1m   Param #\u001B[0m\u001B[1m \u001B[0m┃\u001B[1m \u001B[0m\u001B[1mConnected to     \u001B[0m\u001B[1m \u001B[0m┃\n",
       "┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩\n",
       "│ input_layer_2       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m5000\u001B[0m, \u001B[38;5;34m1\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ -                 │\n",
       "│ (\u001B[38;5;33mInputLayer\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_10    │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m1\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ input_layer_2[\u001B[38;5;34m0\u001B[0m]… │\n",
       "│ (\u001B[38;5;33mMaxPooling1D\u001B[0m)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m1\u001B[0m)   │          \u001B[38;5;34m4\u001B[0m │ max_pooling1d_10… │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_86       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m1\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_93 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m16\u001B[0m │ activation_86[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m64\u001B[0m │ conv1d_93[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_87       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_94 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │        \u001B[38;5;34m768\u001B[0m │ activation_87[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m64\u001B[0m │ conv1d_94[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_88       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_92 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │        \u001B[38;5;34m128\u001B[0m │ activation_86[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_95 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │      \u001B[38;5;34m1,088\u001B[0m │ activation_88[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_28 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ conv1d_92[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m],  │\n",
       "│                     │                   │            │ conv1d_95[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │        \u001B[38;5;34m256\u001B[0m │ add_28[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_89       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_96 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ activation_89[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m64\u001B[0m │ conv1d_96[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_90       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_97 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │        \u001B[38;5;34m768\u001B[0m │ activation_90[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m64\u001B[0m │ conv1d_97[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_91       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_98 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │      \u001B[38;5;34m1,088\u001B[0m │ activation_91[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_29 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_89[\u001B[38;5;34m0\u001B[0m]… │\n",
       "│                     │                   │            │ conv1d_98[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │        \u001B[38;5;34m256\u001B[0m │ add_29[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_92       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m64\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_99 (\u001B[38;5;33mConv1D\u001B[0m)  │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ activation_92[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │         \u001B[38;5;34m64\u001B[0m │ conv1d_99[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_93       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m1250\u001B[0m, \u001B[38;5;34m16\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_100 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m16\u001B[0m)   │        \u001B[38;5;34m768\u001B[0m │ activation_93[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m16\u001B[0m)   │         \u001B[38;5;34m64\u001B[0m │ conv1d_100[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_94       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m16\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_11    │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ activation_92[\u001B[38;5;34m0\u001B[0m]… │\n",
       "│ (\u001B[38;5;33mMaxPooling1D\u001B[0m)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_101 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │      \u001B[38;5;34m1,088\u001B[0m │ activation_94[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_30 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ max_pooling1d_11… │\n",
       "│                     │                   │            │ conv1d_101[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ add_30[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_95       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_103 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m2,048\u001B[0m │ activation_95[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_103[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_96       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_104 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m3,072\u001B[0m │ activation_96[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_104[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_97       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_102 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │      \u001B[38;5;34m8,320\u001B[0m │ activation_95[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_105 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │      \u001B[38;5;34m4,224\u001B[0m │ activation_97[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_31 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ conv1d_102[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m], │\n",
       "│                     │                   │            │ conv1d_105[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ add_31[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_98       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_106 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m4,096\u001B[0m │ activation_98[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_106[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_99       │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_107 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m3,072\u001B[0m │ activation_99[\u001B[38;5;34m0\u001B[0m]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_107[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_100      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_108 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │      \u001B[38;5;34m4,224\u001B[0m │ activation_100[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_32 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_98[\u001B[38;5;34m0\u001B[0m]… │\n",
       "│                     │                   │            │ conv1d_108[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ add_32[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_101      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_109 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m4,096\u001B[0m │ activation_101[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_109[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_102      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_110 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m3,072\u001B[0m │ activation_102[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_110[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_103      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_111 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │      \u001B[38;5;34m4,224\u001B[0m │ activation_103[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_33 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_101[\u001B[38;5;34m0\u001B[0m… │\n",
       "│                     │                   │            │ conv1d_111[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ add_33[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_104      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_112 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m4,096\u001B[0m │ activation_104[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_112[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_105      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m625\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_113 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │      \u001B[38;5;34m3,072\u001B[0m │ activation_105[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │        \u001B[38;5;34m128\u001B[0m │ conv1d_113[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_106      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m32\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_12    │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_104[\u001B[38;5;34m0\u001B[0m… │\n",
       "│ (\u001B[38;5;33mMaxPooling1D\u001B[0m)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_114 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │      \u001B[38;5;34m4,224\u001B[0m │ activation_106[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_34 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ max_pooling1d_12… │\n",
       "│                     │                   │            │ conv1d_114[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ add_34[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_107      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_116 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │      \u001B[38;5;34m8,192\u001B[0m │ activation_107[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_116[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_108      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_117 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m12,288\u001B[0m │ activation_108[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_117[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_109      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_115 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │     \u001B[38;5;34m33,024\u001B[0m │ activation_107[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_118 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │     \u001B[38;5;34m16,640\u001B[0m │ activation_109[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_35 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ conv1d_115[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m], │\n",
       "│                     │                   │            │ conv1d_118[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ add_35[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_110      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_119 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m16,384\u001B[0m │ activation_110[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_119[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_111      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_120 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m12,288\u001B[0m │ activation_111[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_120[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_112      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_121 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │     \u001B[38;5;34m16,640\u001B[0m │ activation_112[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_36 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_110[\u001B[38;5;34m0\u001B[0m… │\n",
       "│                     │                   │            │ conv1d_121[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ add_36[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_113      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_122 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m16,384\u001B[0m │ activation_113[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_122[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_114      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_123 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m12,288\u001B[0m │ activation_114[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_123[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_115      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_124 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │     \u001B[38;5;34m16,640\u001B[0m │ activation_115[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_37 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_113[\u001B[38;5;34m0\u001B[0m… │\n",
       "│                     │                   │            │ conv1d_124[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ add_37[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_116      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_125 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m16,384\u001B[0m │ activation_116[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_125[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_117      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m313\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_126 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │     \u001B[38;5;34m12,288\u001B[0m │ activation_117[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │        \u001B[38;5;34m256\u001B[0m │ conv1d_126[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_118      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m64\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_13    │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_116[\u001B[38;5;34m0\u001B[0m… │\n",
       "│ (\u001B[38;5;33mMaxPooling1D\u001B[0m)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_127 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │     \u001B[38;5;34m16,640\u001B[0m │ activation_118[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_38 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ max_pooling1d_13… │\n",
       "│                     │                   │            │ conv1d_127[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │      \u001B[38;5;34m1,024\u001B[0m │ add_38[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_119      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m256\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_129 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │     \u001B[38;5;34m32,768\u001B[0m │ activation_119[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ conv1d_129[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_120      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_130 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │     \u001B[38;5;34m49,152\u001B[0m │ activation_120[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ conv1d_130[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_121      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_128 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │    \u001B[38;5;34m131,584\u001B[0m │ activation_119[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_131 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │     \u001B[38;5;34m66,048\u001B[0m │ activation_121[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_39 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ conv1d_128[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m], │\n",
       "│                     │                   │            │ conv1d_131[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │      \u001B[38;5;34m2,048\u001B[0m │ add_39[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_122      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_132 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │     \u001B[38;5;34m65,536\u001B[0m │ activation_122[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ conv1d_132[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_123      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_133 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │     \u001B[38;5;34m49,152\u001B[0m │ activation_123[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ conv1d_133[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_124      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_134 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │     \u001B[38;5;34m66,048\u001B[0m │ activation_124[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_40 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ activation_122[\u001B[38;5;34m0\u001B[0m… │\n",
       "│                     │                   │            │ conv1d_134[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │      \u001B[38;5;34m2,048\u001B[0m │ add_40[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_125      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m512\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_135 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │     \u001B[38;5;34m65,536\u001B[0m │ activation_125[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │        \u001B[38;5;34m512\u001B[0m │ conv1d_135[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_126      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m157\u001B[0m, \u001B[38;5;34m128\u001B[0m)  │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_136 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m128\u001B[0m)   │     \u001B[38;5;34m49,152\u001B[0m │ activation_126[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m128\u001B[0m)   │        \u001B[38;5;34m512\u001B[0m │ conv1d_136[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_127      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m128\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_14    │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m512\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ activation_125[\u001B[38;5;34m0\u001B[0m… │\n",
       "│ (\u001B[38;5;33mMaxPooling1D\u001B[0m)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_137 (\u001B[38;5;33mConv1D\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m512\u001B[0m)   │     \u001B[38;5;34m66,048\u001B[0m │ activation_127[\u001B[38;5;34m0\u001B[0m… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_41 (\u001B[38;5;33mAdd\u001B[0m)        │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m79\u001B[0m, \u001B[38;5;34m512\u001B[0m)   │          \u001B[38;5;34m0\u001B[0m │ max_pooling1d_14… │\n",
       "│                     │                   │            │ conv1d_137[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ global_average_poo… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m512\u001B[0m)       │          \u001B[38;5;34m0\u001B[0m │ add_41[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]      │\n",
       "│ (\u001B[38;5;33mGlobalAveragePool…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dropout_2 (\u001B[38;5;33mDropout\u001B[0m) │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m512\u001B[0m)       │          \u001B[38;5;34m0\u001B[0m │ global_average_p… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dense_4 (\u001B[38;5;33mDense\u001B[0m)     │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m256\u001B[0m)       │    \u001B[38;5;34m131,328\u001B[0m │ dropout_2[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m256\u001B[0m)       │      \u001B[38;5;34m1,024\u001B[0m │ dense_4[\u001B[38;5;34m0\u001B[0m][\u001B[38;5;34m0\u001B[0m]     │\n",
       "│ (\u001B[38;5;33mBatchNormalizatio…\u001B[0m │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_128      │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m256\u001B[0m)       │          \u001B[38;5;34m0\u001B[0m │ batch_normalizat… │\n",
       "│ (\u001B[38;5;33mActivation\u001B[0m)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dense_5 (\u001B[38;5;33mDense\u001B[0m)     │ (\u001B[38;5;45mNone\u001B[0m, \u001B[38;5;34m256\u001B[0m)       │     \u001B[38;5;34m65,792\u001B[0m │ activation_128[\u001B[38;5;34m0\u001B[0m… │\n",
       "└─────────────────────┴───────────────────┴────────────┴───────────────────┘\n"
      ],
      "text/html": [
       "<pre style=\"white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace\">┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓\n",
       "┃<span style=\"font-weight: bold\"> Layer (type)        </span>┃<span style=\"font-weight: bold\"> Output Shape      </span>┃<span style=\"font-weight: bold\">    Param # </span>┃<span style=\"font-weight: bold\"> Connected to      </span>┃\n",
       "┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩\n",
       "│ input_layer_2       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">5000</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ -                 │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">InputLayer</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_10    │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ input_layer_2[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">MaxPooling1D</span>)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">4</span> │ max_pooling1d_10… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_86       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_93 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span> │ activation_86[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_93[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_87       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_94 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">768</span> │ activation_87[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_94[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_88       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_92 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ activation_86[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_95 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,088</span> │ activation_88[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_28 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ conv1d_92[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>],  │\n",
       "│                     │                   │            │ conv1d_95[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ add_28[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_89       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_96 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ activation_89[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_96[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_90       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_97 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">768</span> │ activation_90[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_97[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_91       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_98 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,088</span> │ activation_91[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_29 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_89[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "│                     │                   │            │ conv1d_98[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ add_29[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_92       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_99 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>)  │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ activation_92[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_99[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_93       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">1250</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_100 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">768</span> │ activation_93[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)   │         <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span> │ conv1d_100[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_94       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">16</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_11    │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_92[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">MaxPooling1D</span>)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_101 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,088</span> │ activation_94[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_30 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ max_pooling1d_11… │\n",
       "│                     │                   │            │ conv1d_101[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ add_30[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_95       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_103 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">2,048</span> │ activation_95[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_103[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_96       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_104 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">3,072</span> │ activation_96[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_104[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_97       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_102 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">8,320</span> │ activation_95[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_105 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,224</span> │ activation_97[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_31 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ conv1d_102[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>], │\n",
       "│                     │                   │            │ conv1d_105[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ add_31[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_98       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_106 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,096</span> │ activation_98[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_106[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_99       │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_107 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">3,072</span> │ activation_99[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_107[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_100      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_108 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,224</span> │ activation_100[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_32 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_98[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]… │\n",
       "│                     │                   │            │ conv1d_108[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ add_32[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_101      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_109 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,096</span> │ activation_101[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_109[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_102      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_110 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">3,072</span> │ activation_102[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_110[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_103      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_111 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,224</span> │ activation_103[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_33 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_101[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│                     │                   │            │ conv1d_111[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ add_33[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_104      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_112 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,096</span> │ activation_104[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_112[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_105      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">625</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_113 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">3,072</span> │ activation_105[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span> │ conv1d_113[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_106      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">32</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_12    │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_104[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">MaxPooling1D</span>)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_114 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">4,224</span> │ activation_106[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_34 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ max_pooling1d_12… │\n",
       "│                     │                   │            │ conv1d_114[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ add_34[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_107      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_116 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │      <span style=\"color: #00af00; text-decoration-color: #00af00\">8,192</span> │ activation_107[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_116[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_108      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_117 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">12,288</span> │ activation_108[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_117[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_109      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_115 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">33,024</span> │ activation_107[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_118 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,640</span> │ activation_109[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_35 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ conv1d_115[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>], │\n",
       "│                     │                   │            │ conv1d_118[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ add_35[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_110      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_119 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,384</span> │ activation_110[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_119[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_111      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_120 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">12,288</span> │ activation_111[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_120[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_112      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_121 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,640</span> │ activation_112[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_36 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_110[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│                     │                   │            │ conv1d_121[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ add_36[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_113      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_122 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,384</span> │ activation_113[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_122[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_114      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_123 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">12,288</span> │ activation_114[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_123[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_115      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_124 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,640</span> │ activation_115[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_37 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_113[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│                     │                   │            │ conv1d_124[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ add_37[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_116      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_125 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,384</span> │ activation_116[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_125[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_117      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">313</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_126 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">12,288</span> │ activation_117[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span> │ conv1d_126[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_118      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">64</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_13    │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_116[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">MaxPooling1D</span>)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_127 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">16,640</span> │ activation_118[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_38 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ max_pooling1d_13… │\n",
       "│                     │                   │            │ conv1d_127[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ add_38[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_119      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_129 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">32,768</span> │ activation_119[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_129[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_120      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_130 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">49,152</span> │ activation_120[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_130[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_121      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_128 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │    <span style=\"color: #00af00; text-decoration-color: #00af00\">131,584</span> │ activation_119[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_131 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">66,048</span> │ activation_121[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_39 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ conv1d_128[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>], │\n",
       "│                     │                   │            │ conv1d_131[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">2,048</span> │ add_39[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_122      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_132 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">65,536</span> │ activation_122[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_132[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_123      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_133 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">49,152</span> │ activation_123[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_133[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_124      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_134 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">66,048</span> │ activation_124[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_40 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_122[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│                     │                   │            │ conv1d_134[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │      <span style=\"color: #00af00; text-decoration-color: #00af00\">2,048</span> │ add_40[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_125      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_135 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │     <span style=\"color: #00af00; text-decoration-color: #00af00\">65,536</span> │ activation_125[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_135[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_126      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">157</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)  │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_136 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">49,152</span> │ activation_126[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)   │        <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span> │ conv1d_136[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_127      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">128</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ max_pooling1d_14    │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ activation_125[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">MaxPooling1D</span>)      │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ conv1d_137 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Conv1D</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)   │     <span style=\"color: #00af00; text-decoration-color: #00af00\">66,048</span> │ activation_127[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ add_41 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Add</span>)        │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">79</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)   │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ max_pooling1d_14… │\n",
       "│                     │                   │            │ conv1d_137[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]  │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ global_average_poo… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)       │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ add_41[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]      │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">GlobalAveragePool…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dropout_2 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Dropout</span>) │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">512</span>)       │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ global_average_p… │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dense_4 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Dense</span>)     │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)       │    <span style=\"color: #00af00; text-decoration-color: #00af00\">131,328</span> │ dropout_2[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ batch_normalizatio… │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)       │      <span style=\"color: #00af00; text-decoration-color: #00af00\">1,024</span> │ dense_4[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>][<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>]     │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">BatchNormalizatio…</span> │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ activation_128      │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)       │          <span style=\"color: #00af00; text-decoration-color: #00af00\">0</span> │ batch_normalizat… │\n",
       "│ (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Activation</span>)        │                   │            │                   │\n",
       "├─────────────────────┼───────────────────┼────────────┼───────────────────┤\n",
       "│ dense_5 (<span style=\"color: #0087ff; text-decoration-color: #0087ff\">Dense</span>)     │ (<span style=\"color: #00d7ff; text-decoration-color: #00d7ff\">None</span>, <span style=\"color: #00af00; text-decoration-color: #00af00\">256</span>)       │     <span style=\"color: #00af00; text-decoration-color: #00af00\">65,792</span> │ activation_128[<span style=\"color: #00af00; text-decoration-color: #00af00\">0</span>… │\n",
       "└─────────────────────┴───────────────────┴────────────┴───────────────────┘\n",
       "</pre>\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "\u001B[1m Total params: \u001B[0m\u001B[38;5;34m1,122,388\u001B[0m (4.28 MB)\n"
      ],
      "text/html": [
       "<pre style=\"white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace\"><span style=\"font-weight: bold\"> Total params: </span><span style=\"color: #00af00; text-decoration-color: #00af00\">1,122,388</span> (4.28 MB)\n",
       "</pre>\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "\u001B[1m Trainable params: \u001B[0m\u001B[38;5;34m1,113,106\u001B[0m (4.25 MB)\n"
      ],
      "text/html": [
       "<pre style=\"white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace\"><span style=\"font-weight: bold\"> Trainable params: </span><span style=\"color: #00af00; text-decoration-color: #00af00\">1,113,106</span> (4.25 MB)\n",
       "</pre>\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "\u001B[1m Non-trainable params: \u001B[0m\u001B[38;5;34m9,282\u001B[0m (36.26 KB)\n"
      ],
      "text/html": [
       "<pre style=\"white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace\"><span style=\"font-weight: bold\"> Non-trainable params: </span><span style=\"color: #00af00; text-decoration-color: #00af00\">9,282</span> (36.26 KB)\n",
       "</pre>\n"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "execution_count": 75
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "Train it on the correct data, kan være lurt å gjøre før man skal predicte...",
   "id": "1b6f7558a363a331"
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T15:06:29.822597Z",
     "start_time": "2025-03-13T15:06:29.819137Z"
    }
   },
   "cell_type": "code",
   "source": "input_shape = x_train.shape[1:]",
   "id": "8ef705cdec76a9f0",
   "outputs": [],
   "execution_count": 104
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T15:15:11.421049Z",
     "start_time": "2025-03-13T15:10:34.709140Z"
    }
   },
   "cell_type": "code",
   "source": [
    "\n",
    "from keras.callbacks import ModelCheckpoint, TensorBoard\n",
    "model_name = 'sub_bytes_in'\n",
    "# Definer callback-listen\n",
    "cb = [\n",
    "    ModelCheckpoint(filepath=model_name +\".keras\", monitor=\"val_loss\", mode=\"min\", save_best_only=True),\n",
    "    TensorBoard(log_dir=\"logs/\", update_freq=\"batch\")\n",
    "]\n",
    "\n",
    "# Tren modellen\n",
    "history = sub_in_0_byte_cnn_scaaml.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=1, epochs=32, callbacks=cb)\n"
   ],
   "id": "4dc974ce0dc62d32",
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/32\n",
      "\u001B[1m 677/2048\u001B[0m \u001B[32m━━━━━━\u001B[0m\u001B[37m━━━━━━━━━━━━━━\u001B[0m \u001B[1m9:18\u001B[0m 407ms/step - acc: 0.0101 - loss: 5.4010"
     ]
    },
    {
     "ename": "KeyboardInterrupt",
     "evalue": "",
     "output_type": "error",
     "traceback": [
      "\u001B[31m---------------------------------------------------------------------------\u001B[39m",
      "\u001B[31mKeyboardInterrupt\u001B[39m                         Traceback (most recent call last)",
      "\u001B[36mCell\u001B[39m\u001B[36m \u001B[39m\u001B[32mIn[112]\u001B[39m\u001B[32m, line 10\u001B[39m\n\u001B[32m      4\u001B[39m cb = [\n\u001B[32m      5\u001B[39m     ModelCheckpoint(filepath=\u001B[33m\"\u001B[39m\u001B[33mbest_model.h5\u001B[39m\u001B[33m\"\u001B[39m, monitor=\u001B[33m\"\u001B[39m\u001B[33mval_loss\u001B[39m\u001B[33m\"\u001B[39m, save_best_only=\u001B[38;5;28;01mTrue\u001B[39;00m),\n\u001B[32m      6\u001B[39m     TensorBoard(log_dir=\u001B[33m\"\u001B[39m\u001B[33mlogs/\u001B[39m\u001B[33m\"\u001B[39m, update_freq=\u001B[33m\"\u001B[39m\u001B[33mbatch\u001B[39m\u001B[33m\"\u001B[39m)\n\u001B[32m      7\u001B[39m ]\n\u001B[32m      9\u001B[39m \u001B[38;5;66;03m# Tren modellen\u001B[39;00m\n\u001B[32m---> \u001B[39m\u001B[32m10\u001B[39m history = \u001B[43msub_in_0_byte_cnn_scaaml\u001B[49m\u001B[43m.\u001B[49m\u001B[43mfit\u001B[49m\u001B[43m(\u001B[49m\u001B[43mx_train\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43my_train\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mvalidation_data\u001B[49m\u001B[43m=\u001B[49m\u001B[43m(\u001B[49m\u001B[43mx_test\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43my_test\u001B[49m\u001B[43m)\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mverbose\u001B[49m\u001B[43m=\u001B[49m\u001B[32;43m1\u001B[39;49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mepochs\u001B[49m\u001B[43m=\u001B[49m\u001B[32;43m32\u001B[39;49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mcallbacks\u001B[49m\u001B[43m=\u001B[49m\u001B[43mcb\u001B[49m\u001B[43m)\u001B[49m\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\keras\\src\\utils\\traceback_utils.py:117\u001B[39m, in \u001B[36mfilter_traceback.<locals>.error_handler\u001B[39m\u001B[34m(*args, **kwargs)\u001B[39m\n\u001B[32m    115\u001B[39m filtered_tb = \u001B[38;5;28;01mNone\u001B[39;00m\n\u001B[32m    116\u001B[39m \u001B[38;5;28;01mtry\u001B[39;00m:\n\u001B[32m--> \u001B[39m\u001B[32m117\u001B[39m     \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[43mfn\u001B[49m\u001B[43m(\u001B[49m\u001B[43m*\u001B[49m\u001B[43margs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43m*\u001B[49m\u001B[43m*\u001B[49m\u001B[43mkwargs\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    118\u001B[39m \u001B[38;5;28;01mexcept\u001B[39;00m \u001B[38;5;167;01mException\u001B[39;00m \u001B[38;5;28;01mas\u001B[39;00m e:\n\u001B[32m    119\u001B[39m     filtered_tb = _process_traceback_frames(e.__traceback__)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\keras\\src\\backend\\tensorflow\\trainer.py:371\u001B[39m, in \u001B[36mTensorFlowTrainer.fit\u001B[39m\u001B[34m(self, x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps, validation_batch_size, validation_freq)\u001B[39m\n\u001B[32m    369\u001B[39m \u001B[38;5;28;01mfor\u001B[39;00m step, iterator \u001B[38;5;129;01min\u001B[39;00m epoch_iterator:\n\u001B[32m    370\u001B[39m     callbacks.on_train_batch_begin(step)\n\u001B[32m--> \u001B[39m\u001B[32m371\u001B[39m     logs = \u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43mtrain_function\u001B[49m\u001B[43m(\u001B[49m\u001B[43miterator\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    372\u001B[39m     callbacks.on_train_batch_end(step, logs)\n\u001B[32m    373\u001B[39m     \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m.stop_training:\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\keras\\src\\backend\\tensorflow\\trainer.py:219\u001B[39m, in \u001B[36mTensorFlowTrainer._make_function.<locals>.function\u001B[39m\u001B[34m(iterator)\u001B[39m\n\u001B[32m    215\u001B[39m \u001B[38;5;28;01mdef\u001B[39;00m\u001B[38;5;250m \u001B[39m\u001B[34mfunction\u001B[39m(iterator):\n\u001B[32m    216\u001B[39m     \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28misinstance\u001B[39m(\n\u001B[32m    217\u001B[39m         iterator, (tf.data.Iterator, tf.distribute.DistributedIterator)\n\u001B[32m    218\u001B[39m     ):\n\u001B[32m--> \u001B[39m\u001B[32m219\u001B[39m         opt_outputs = \u001B[43mmulti_step_on_iterator\u001B[49m\u001B[43m(\u001B[49m\u001B[43miterator\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    220\u001B[39m         \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;129;01mnot\u001B[39;00m opt_outputs.has_value():\n\u001B[32m    221\u001B[39m             \u001B[38;5;28;01mraise\u001B[39;00m \u001B[38;5;167;01mStopIteration\u001B[39;00m\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\util\\traceback_utils.py:150\u001B[39m, in \u001B[36mfilter_traceback.<locals>.error_handler\u001B[39m\u001B[34m(*args, **kwargs)\u001B[39m\n\u001B[32m    148\u001B[39m filtered_tb = \u001B[38;5;28;01mNone\u001B[39;00m\n\u001B[32m    149\u001B[39m \u001B[38;5;28;01mtry\u001B[39;00m:\n\u001B[32m--> \u001B[39m\u001B[32m150\u001B[39m   \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[43mfn\u001B[49m\u001B[43m(\u001B[49m\u001B[43m*\u001B[49m\u001B[43margs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43m*\u001B[49m\u001B[43m*\u001B[49m\u001B[43mkwargs\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    151\u001B[39m \u001B[38;5;28;01mexcept\u001B[39;00m \u001B[38;5;167;01mException\u001B[39;00m \u001B[38;5;28;01mas\u001B[39;00m e:\n\u001B[32m    152\u001B[39m   filtered_tb = _process_traceback_frames(e.__traceback__)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\polymorphic_function.py:833\u001B[39m, in \u001B[36mFunction.__call__\u001B[39m\u001B[34m(self, *args, **kwds)\u001B[39m\n\u001B[32m    830\u001B[39m compiler = \u001B[33m\"\u001B[39m\u001B[33mxla\u001B[39m\u001B[33m\"\u001B[39m \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m._jit_compile \u001B[38;5;28;01melse\u001B[39;00m \u001B[33m\"\u001B[39m\u001B[33mnonXla\u001B[39m\u001B[33m\"\u001B[39m\n\u001B[32m    832\u001B[39m \u001B[38;5;28;01mwith\u001B[39;00m OptionalXlaContext(\u001B[38;5;28mself\u001B[39m._jit_compile):\n\u001B[32m--> \u001B[39m\u001B[32m833\u001B[39m   result = \u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43m_call\u001B[49m\u001B[43m(\u001B[49m\u001B[43m*\u001B[49m\u001B[43margs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43m*\u001B[49m\u001B[43m*\u001B[49m\u001B[43mkwds\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    835\u001B[39m new_tracing_count = \u001B[38;5;28mself\u001B[39m.experimental_get_tracing_count()\n\u001B[32m    836\u001B[39m without_tracing = (tracing_count == new_tracing_count)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\polymorphic_function.py:878\u001B[39m, in \u001B[36mFunction._call\u001B[39m\u001B[34m(self, *args, **kwds)\u001B[39m\n\u001B[32m    875\u001B[39m \u001B[38;5;28mself\u001B[39m._lock.release()\n\u001B[32m    876\u001B[39m \u001B[38;5;66;03m# In this case we have not created variables on the first call. So we can\u001B[39;00m\n\u001B[32m    877\u001B[39m \u001B[38;5;66;03m# run the first trace but we should fail if variables are created.\u001B[39;00m\n\u001B[32m--> \u001B[39m\u001B[32m878\u001B[39m results = \u001B[43mtracing_compilation\u001B[49m\u001B[43m.\u001B[49m\u001B[43mcall_function\u001B[49m\u001B[43m(\u001B[49m\n\u001B[32m    879\u001B[39m \u001B[43m    \u001B[49m\u001B[43margs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mkwds\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43m_variable_creation_config\u001B[49m\n\u001B[32m    880\u001B[39m \u001B[43m\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    881\u001B[39m \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m._created_variables:\n\u001B[32m    882\u001B[39m   \u001B[38;5;28;01mraise\u001B[39;00m \u001B[38;5;167;01mValueError\u001B[39;00m(\u001B[33m\"\u001B[39m\u001B[33mCreating variables on a non-first call to a function\u001B[39m\u001B[33m\"\u001B[39m\n\u001B[32m    883\u001B[39m                    \u001B[33m\"\u001B[39m\u001B[33m decorated with tf.function.\u001B[39m\u001B[33m\"\u001B[39m)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\tracing_compilation.py:139\u001B[39m, in \u001B[36mcall_function\u001B[39m\u001B[34m(args, kwargs, tracing_options)\u001B[39m\n\u001B[32m    137\u001B[39m bound_args = function.function_type.bind(*args, **kwargs)\n\u001B[32m    138\u001B[39m flat_inputs = function.function_type.unpack_inputs(bound_args)\n\u001B[32m--> \u001B[39m\u001B[32m139\u001B[39m \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[43mfunction\u001B[49m\u001B[43m.\u001B[49m\u001B[43m_call_flat\u001B[49m\u001B[43m(\u001B[49m\u001B[43m  \u001B[49m\u001B[38;5;66;43;03m# pylint: disable=protected-access\u001B[39;49;00m\n\u001B[32m    140\u001B[39m \u001B[43m    \u001B[49m\u001B[43mflat_inputs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mcaptured_inputs\u001B[49m\u001B[43m=\u001B[49m\u001B[43mfunction\u001B[49m\u001B[43m.\u001B[49m\u001B[43mcaptured_inputs\u001B[49m\n\u001B[32m    141\u001B[39m \u001B[43m\u001B[49m\u001B[43m)\u001B[49m\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\concrete_function.py:1322\u001B[39m, in \u001B[36mConcreteFunction._call_flat\u001B[39m\u001B[34m(self, tensor_inputs, captured_inputs)\u001B[39m\n\u001B[32m   1318\u001B[39m possible_gradient_type = gradients_util.PossibleTapeGradientTypes(args)\n\u001B[32m   1319\u001B[39m \u001B[38;5;28;01mif\u001B[39;00m (possible_gradient_type == gradients_util.POSSIBLE_GRADIENT_TYPES_NONE\n\u001B[32m   1320\u001B[39m     \u001B[38;5;129;01mand\u001B[39;00m executing_eagerly):\n\u001B[32m   1321\u001B[39m   \u001B[38;5;66;03m# No tape is watching; skip to running the function.\u001B[39;00m\n\u001B[32m-> \u001B[39m\u001B[32m1322\u001B[39m   \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43m_inference_function\u001B[49m\u001B[43m.\u001B[49m\u001B[43mcall_preflattened\u001B[49m\u001B[43m(\u001B[49m\u001B[43margs\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m   1323\u001B[39m forward_backward = \u001B[38;5;28mself\u001B[39m._select_forward_and_backward_functions(\n\u001B[32m   1324\u001B[39m     args,\n\u001B[32m   1325\u001B[39m     possible_gradient_type,\n\u001B[32m   1326\u001B[39m     executing_eagerly)\n\u001B[32m   1327\u001B[39m forward_function, args_with_tangents = forward_backward.forward()\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\atomic_function.py:216\u001B[39m, in \u001B[36mAtomicFunction.call_preflattened\u001B[39m\u001B[34m(self, args)\u001B[39m\n\u001B[32m    214\u001B[39m \u001B[38;5;28;01mdef\u001B[39;00m\u001B[38;5;250m \u001B[39m\u001B[34mcall_preflattened\u001B[39m(\u001B[38;5;28mself\u001B[39m, args: Sequence[core.Tensor]) -> Any:\n\u001B[32m    215\u001B[39m \u001B[38;5;250m  \u001B[39m\u001B[33;03m\"\"\"Calls with flattened tensor inputs and returns the structured output.\"\"\"\u001B[39;00m\n\u001B[32m--> \u001B[39m\u001B[32m216\u001B[39m   flat_outputs = \u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43mcall_flat\u001B[49m\u001B[43m(\u001B[49m\u001B[43m*\u001B[49m\u001B[43margs\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    217\u001B[39m   \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28mself\u001B[39m.function_type.pack_output(flat_outputs)\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\polymorphic_function\\atomic_function.py:251\u001B[39m, in \u001B[36mAtomicFunction.call_flat\u001B[39m\u001B[34m(self, *args)\u001B[39m\n\u001B[32m    249\u001B[39m \u001B[38;5;28;01mwith\u001B[39;00m record.stop_recording():\n\u001B[32m    250\u001B[39m   \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m._bound_context.executing_eagerly():\n\u001B[32m--> \u001B[39m\u001B[32m251\u001B[39m     outputs = \u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43m_bound_context\u001B[49m\u001B[43m.\u001B[49m\u001B[43mcall_function\u001B[49m\u001B[43m(\u001B[49m\n\u001B[32m    252\u001B[39m \u001B[43m        \u001B[49m\u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43mname\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m    253\u001B[39m \u001B[43m        \u001B[49m\u001B[38;5;28;43mlist\u001B[39;49m\u001B[43m(\u001B[49m\u001B[43margs\u001B[49m\u001B[43m)\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m    254\u001B[39m \u001B[43m        \u001B[49m\u001B[38;5;28;43mlen\u001B[39;49m\u001B[43m(\u001B[49m\u001B[38;5;28;43mself\u001B[39;49m\u001B[43m.\u001B[49m\u001B[43mfunction_type\u001B[49m\u001B[43m.\u001B[49m\u001B[43mflat_outputs\u001B[49m\u001B[43m)\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m    255\u001B[39m \u001B[43m    \u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m    256\u001B[39m   \u001B[38;5;28;01melse\u001B[39;00m:\n\u001B[32m    257\u001B[39m     outputs = make_call_op_in_graph(\n\u001B[32m    258\u001B[39m         \u001B[38;5;28mself\u001B[39m,\n\u001B[32m    259\u001B[39m         \u001B[38;5;28mlist\u001B[39m(args),\n\u001B[32m    260\u001B[39m         \u001B[38;5;28mself\u001B[39m._bound_context.function_call_options.as_attrs(),\n\u001B[32m    261\u001B[39m     )\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\context.py:1683\u001B[39m, in \u001B[36mContext.call_function\u001B[39m\u001B[34m(self, name, tensor_inputs, num_outputs)\u001B[39m\n\u001B[32m   1681\u001B[39m cancellation_context = cancellation.context()\n\u001B[32m   1682\u001B[39m \u001B[38;5;28;01mif\u001B[39;00m cancellation_context \u001B[38;5;129;01mis\u001B[39;00m \u001B[38;5;28;01mNone\u001B[39;00m:\n\u001B[32m-> \u001B[39m\u001B[32m1683\u001B[39m   outputs = \u001B[43mexecute\u001B[49m\u001B[43m.\u001B[49m\u001B[43mexecute\u001B[49m\u001B[43m(\u001B[49m\n\u001B[32m   1684\u001B[39m \u001B[43m      \u001B[49m\u001B[43mname\u001B[49m\u001B[43m.\u001B[49m\u001B[43mdecode\u001B[49m\u001B[43m(\u001B[49m\u001B[33;43m\"\u001B[39;49m\u001B[33;43mutf-8\u001B[39;49m\u001B[33;43m\"\u001B[39;49m\u001B[43m)\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m   1685\u001B[39m \u001B[43m      \u001B[49m\u001B[43mnum_outputs\u001B[49m\u001B[43m=\u001B[49m\u001B[43mnum_outputs\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m   1686\u001B[39m \u001B[43m      \u001B[49m\u001B[43minputs\u001B[49m\u001B[43m=\u001B[49m\u001B[43mtensor_inputs\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m   1687\u001B[39m \u001B[43m      \u001B[49m\u001B[43mattrs\u001B[49m\u001B[43m=\u001B[49m\u001B[43mattrs\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m   1688\u001B[39m \u001B[43m      \u001B[49m\u001B[43mctx\u001B[49m\u001B[43m=\u001B[49m\u001B[38;5;28;43mself\u001B[39;49m\u001B[43m,\u001B[49m\n\u001B[32m   1689\u001B[39m \u001B[43m  \u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m   1690\u001B[39m \u001B[38;5;28;01melse\u001B[39;00m:\n\u001B[32m   1691\u001B[39m   outputs = execute.execute_with_cancellation(\n\u001B[32m   1692\u001B[39m       name.decode(\u001B[33m\"\u001B[39m\u001B[33mutf-8\u001B[39m\u001B[33m\"\u001B[39m),\n\u001B[32m   1693\u001B[39m       num_outputs=num_outputs,\n\u001B[32m   (...)\u001B[39m\u001B[32m   1697\u001B[39m       cancellation_manager=cancellation_context,\n\u001B[32m   1698\u001B[39m   )\n",
      "\u001B[36mFile \u001B[39m\u001B[32m~\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\python\\eager\\execute.py:53\u001B[39m, in \u001B[36mquick_execute\u001B[39m\u001B[34m(op_name, num_outputs, inputs, attrs, ctx, name)\u001B[39m\n\u001B[32m     51\u001B[39m \u001B[38;5;28;01mtry\u001B[39;00m:\n\u001B[32m     52\u001B[39m   ctx.ensure_initialized()\n\u001B[32m---> \u001B[39m\u001B[32m53\u001B[39m   tensors = \u001B[43mpywrap_tfe\u001B[49m\u001B[43m.\u001B[49m\u001B[43mTFE_Py_Execute\u001B[49m\u001B[43m(\u001B[49m\u001B[43mctx\u001B[49m\u001B[43m.\u001B[49m\u001B[43m_handle\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mdevice_name\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mop_name\u001B[49m\u001B[43m,\u001B[49m\n\u001B[32m     54\u001B[39m \u001B[43m                                      \u001B[49m\u001B[43minputs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mattrs\u001B[49m\u001B[43m,\u001B[49m\u001B[43m \u001B[49m\u001B[43mnum_outputs\u001B[49m\u001B[43m)\u001B[49m\n\u001B[32m     55\u001B[39m \u001B[38;5;28;01mexcept\u001B[39;00m core._NotOkStatusException \u001B[38;5;28;01mas\u001B[39;00m e:\n\u001B[32m     56\u001B[39m   \u001B[38;5;28;01mif\u001B[39;00m name \u001B[38;5;129;01mis\u001B[39;00m \u001B[38;5;129;01mnot\u001B[39;00m \u001B[38;5;28;01mNone\u001B[39;00m:\n",
      "\u001B[31mKeyboardInterrupt\u001B[39m: "
     ]
    }
   ],
   "execution_count": 112
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "We need to load our attack traces, they are close to the test and train data, but the key is random every time. ATM we do not control for repeat keys!",
   "id": "c470c88e22498d2a"
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:45:55.701564Z",
     "start_time": "2025-03-13T14:45:55.695324Z"
    }
   },
   "cell_type": "code",
   "source": [
    "import numpy.typing as npt\n",
    "\n",
    "def ap_preds_to_key_preds(preds: npt.NDArray[np.generic],\n",
    "                          plaintexts: npt.NDArray[np.generic],\n",
    "                          attack_point: str) -> npt.NDArray[np.generic]:\n",
    "    \"Convert attack points predictions to key byte prediction\"\n",
    "\n",
    "    preds = np.array(preds)\n",
    "\n",
    "    if attack_point == \"key\":\n",
    "        return preds\n",
    "\n",
    "    elif attack_point == \"sub_bytes_in\":\n",
    "        key_probas = []\n",
    "        class_ids = np.arange(256)\n",
    "        for idx, pred in enumerate(preds):\n",
    "            kv = class_ids ^ plaintexts[idx]\n",
    "            key_probas.append(np.take(pred, kv))\n",
    "        return np.array(key_probas)\n",
    "\n",
    "    elif attack_point == \"sub_bytes_out\":\n",
    "        key_probas = []\n",
    "        for idx, pred in enumerate(preds):\n",
    "            # ! danger zone. change at your own peril, Here be dragons.\n",
    "            kidxs = SBREV ^ plaintexts[idx]\n",
    "            probas = np.zeros(256)\n",
    "            for pidx, kidx in enumerate(kidxs):\n",
    "                probas[kidx] = pred[pidx]\n",
    "            key_probas.append(probas)\n",
    "\n",
    "        return np.array(key_probas)\n",
    "    else:\n",
    "        raise ValueError(\"Invalid attack point\")\n",
    "\n",
    "def from_categorical(predictions: Sequence[npt.ArrayLike]) -> List[np.intp]:\n",
    "    \"reverse of categorical\"\n",
    "    # note: doing it as a list is significantly faster than a single argmax\n",
    "    return [np.argmax(p) for p in predictions]"
   ],
   "id": "44c62d0b2d88efd",
   "outputs": [],
   "execution_count": 91
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "Test it\n",
   "id": "a4564cfb7841ac37"
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:59:44.126079Z",
     "start_time": "2025-03-13T14:59:41.959147Z"
    }
   },
   "cell_type": "code",
   "source": [
    "from tensorflow.keras import metrics\n",
    "from tqdm import tqdm\n",
    "arr = np.arange(256)\n",
    "\n",
    "NUM_TRACES = 10  # maximum number of traces to use to recover a given key byte. 10 is already overkill\n",
    "correct_prediction_rank = defaultdict(list)\n",
    "y_pred = []\n",
    "y_true = []\n",
    "model_metrics = {\"acc\": metrics.Accuracy()}\n",
    "#for shard in tqdm(arr, desc='Recovering bytes', unit='shards'):\n",
    "   # keys, pts, x, y = load_shard(shard, ATTACK_BYTE, ATTACK_POINT, TRACE_LEN, num_traces=NUM_TRACES)\n",
    "dataset_name = \"attack\"\n",
    "shard = []\n",
    "attack_byte = 0\n",
    "attack_point = \"sub_bytes_in\"\n",
    "max_trace_length = 5000\n",
    "num_traces = 10\n",
    "\n",
    "full_key = False\n",
    "x_List: List[Tensor] = []\n",
    "y_List: List[Tensor] = []\n",
    "print(\"hello\")\n",
    "    #need to open the file to read any info\n",
    "with (h5py.File(f\"{dataset_name}.hdf5\", \"r\")) as f:\n",
    "    print(\"opening file\")\n",
    "    i = 0\n",
    "    for group in f.keys():\n",
    "\n",
    "        group_name = f[group]\n",
    "        print(\"reading group\")\n",
    "\n",
    "        x_shard = group_name[\"traces\"][:num_traces, :5000, :]\n",
    "        x_shard = tf.convert_to_tensor(x_shard, dtype=\"float32\")\n",
    "        print(\"done with x\")\n",
    "        y_shard = group_name[\"sub_bytes_in\"][attack_byte]\n",
    "        y_shard = y_shard[:num_traces]\n",
    "        y_shard = tf.keras.utils.to_categorical(y_shard, 256)\n",
    "        y_shard = tf.convert_to_tensor(y_shard, dtype=\"uint8\")\n",
    "        print(\"done with y\")\n",
    "\n",
    "        k = group_name[\"key\"][attack_byte][:num_traces]\n",
    "        print(\"done with k\")\n",
    "\n",
    "        pts = group_name[\"pts\"][attack_byte][:num_traces]\n",
    "        print(\"done with pts\")\n",
    "\n",
    "                # prediction\n",
    "        predictions = sub_in_0_byte_cnn_scaaml.predict(x_shard)\n",
    "        print(predictions.shape)\n",
    "                # computing byte prediction from intermediate predictions\n",
    "        key_preds = ap_preds_to_key_preds(predictions, pts, attack_point)\n",
    "\n",
    "        c_preds = from_categorical(predictions)\n",
    "        c_y = from_categorical(y_shard)\n",
    "                # metric tracking\n",
    "        for metric in model_metrics.values():\n",
    "            metric.update_state(c_y, c_preds)\n",
    "                # for the confusion matrix\n",
    "        y_pred.extend(c_preds)\n",
    "        y_true.extend(c_y)\n",
    "\n",
    "            # accumulating probabilities and checking correct guess position.\n",
    "            # if all goes well it will be at position 0 (highest probability)\n",
    "            # see below on how to use for the real attack\n",
    "\n",
    "\n",
    "        key = k[0] # all the same in the same shard - not used in real attack\n",
    "        vals = np.zeros((256))\n",
    "        for trace_count, kp in enumerate(key_preds):\n",
    "            vals = vals  + np.log10(kp + 1e-22)\n",
    "            guess_ranks = (np.argsort(vals, )[-256:][::-1])\n",
    "            byte_rank = list(guess_ranks).index(key)\n",
    "            correct_prediction_rank[trace_count].append(byte_rank)\n"
   ],
   "id": "ccd3cc9e6a2aa0f4",
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "hello\n",
      "opening file\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 104ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 101ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 106ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 99ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 98ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 94ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 95ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 94ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 95ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 94ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 95ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 94ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 97ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 91ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 90ms/step\n",
      "(10, 256)\n",
      "reading group\n",
      "done with x\n",
      "done with y\n",
      "done with k\n",
      "done with pts\n",
      "\u001B[1m1/1\u001B[0m \u001B[32m━━━━━━━━━━━━━━━━━━━━\u001B[0m\u001B[37m\u001B[0m \u001B[1m0s\u001B[0m 90ms/step\n",
      "(10, 256)\n"
     ]
    }
   ],
   "execution_count": 98
  },
  {
   "metadata": {
    "ExecuteTime": {
     "end_time": "2025-03-13T14:59:54.735578Z",
     "start_time": "2025-03-13T14:59:54.721154Z"
    }
   },
   "cell_type": "code",
   "source": "print(\"Accuracy: %.2f\" % model_metrics['acc'].result())\n",
   "id": "a9da887dfb031595",
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Accuracy: 0.00\n"
     ]
    }
   ],
   "execution_count": 99
  },
  {
   "metadata": {},
   "cell_type": "markdown",
   "source": "Fetching info from a attack set will look like this",
   "id": "1a8c6731edcea0e0"
  },
  {
   "metadata": {},
   "cell_type": "code",
   "outputs": [],
   "execution_count": null,
   "source": "",
   "id": "749c3b8efdfbe30a"
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 2
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython2",
   "version": "2.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'true' is not defined